### Grassland Management Under Climate Change

#### A workbook for Sorghastrum nutans's habitat suitability model. 

In this project I will analyze multiple data layers that relate to Sorghastrum nutans habitat in North America. I will focus on USFS National Grassland Units as my study sites to better understand the link between habitat and variables like soil, topography, and climate. The Forest Service defines 20 National Grasslands across 13 different states that provide ecosystem services to over a million visitors annually.Value estimates of aesthetic and passive use of such ecosystems total millions of dollars (~280) yearly. Climate change threatens the continued suitability of certain species that play an important role in preserving grassland integrity and valuable ecosystem servies.  
[(USFS)](https://www.fs.usda.gov/managing-land/national-forests-grasslands/national-grasslands/ecoservices).

In this workbook, I chose Oglala National Grassland (NE) and Fort Pierre National Grassland (SD). My decision was based on comparable shape (contiguous not fragmented), location (SD-NE), acreage (94,480 vs 115,997).
- Oglala National Grassland is a unique area created by water and wind erosion on layers of siltstone and clay. The area is dominated by native mixed-grass prairie[(USFS)](https://www.fs.usda.gov/recarea/nebraska/recarea/?recid=30328).
- Fort Pierre National Grassland consists of flats and gently rolling hills with large drainages that are essential for waterfowl habitat and popular fisheries. Furthermore, it is widely known for its mixed grass prairie vegetation [(USFS)](https://www.fs.usda.gov/recarea/nebraska/recarea/?recid=10637).



In [24]:
# Import packages used for the analysis

import os
from glob import glob

import earthpy as et
import earthpy.appeears as eaapp
import geopandas as gpd
import requests
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr

#### Study Area

A map service depicting National Grassland units designated by the Secretary of Agriculture and permanently held by the Department of Agriculture under Title III of the Bankhead-Jones Farm Tenant Act. Current boundary information. The map service uses a File Geodatabase data source with Web Mercator projection.
[(FSGeodata Clearinghouse)](https://data.fs.usda.gov/geodata/edw/datasets.php?dsetCategory=boundaries)


In [4]:
# Download USFS National Grassland Units and choose grasslands.

grasslands_url = (
    "https://data.fs.usda.gov/geodata/edw/edw_resources/shp"
    "/S_USA.NationalGrassland.zip")
grasslands_gdf = gpd.read_file(grasslands_url)
grasslands_gdf

# Select two grasslands
# Selection criteria: comparable shape, area, "similar" location
grass_gdf = (
    grasslands_gdf
    .set_index('GRASSLANDN')
    .loc[['Oglala National Grassland', 'Fort Pierre National Grassland']]
)

grass_gdf

for GRASSLANDN, grass_gs in grass_gdf.iterrows():
    print('\nGrassland name: {}'.format(GRASSLANDN))
    print(grass_gs)
    print(f"Total bounds: {grass_gs.geometry.bounds}")


Grassland name: Oglala National Grassland
NATIONALGR                                         295521010328
GIS_ACRES                                            215804.927
SHAPE_AREA                                             0.096279
SHAPE_LEN                                              1.970612
geometry      POLYGON ((-103.72476827000003 43.0009967600000...
Name: Oglala National Grassland, dtype: object
Total bounds: (-104.05313982000001, 42.74092908, -103.36517901000002, 43.00177636000001)

Grassland name: Fort Pierre National Grassland
NATIONALGR                                         281771010328
GIS_ACRES                                            209044.225
SHAPE_AREA                                             0.095149
SHAPE_LEN                                              1.455518
geometry      POLYGON ((-100.08408953000003 44.2816249699999...
Name: Fort Pierre National Grassland, dtype: object
Total bounds: (-100.47626474999998, 43.977281059999996, -100.06625771, 44.2816249

#### Polaris Soil Dataset

[Polaris](http://hydrology.cee.duke.edu/POLARIS) is constructed using available high-resolution geospatial environmental data and a state-of-the-art machine learning algorithm (DSMART-HPC) to remap the Soil Survey Geographic (SSURGO) database. This 9 billion grid cell database is possible using available high performance computing resources. POLARIS provides a spatially continuous, internally consistent, quantitative prediction of soil series.
(Chaney, N. W., Wood, E. F., McBratney, A. B., Hempel, J. W., Nauman, T. W., Brungard, C. W., & Odgers, N. P. (2016). POLARIS: A \30-meter probabilistic soil series map of the contiguous United States. Geoderma, 274, 54-67.)

In [55]:
# Download /POLARIS/PROPERTIES/v1.0/ph/mean/60_100
# Soil variable and change tiles according to chosen grassland units 
# Unit might not fit into one tile: 
# load in more than one tile per grassland 
# and merge with rioxarray

# Probably I need a function (only loop?)
# since I have two different 
# set of min values (from total bounds)

min_lat, min_lon = 42, -104
max_lat, max_lon = min_lat+1, min_lon+1
polaris_url_format = (
    "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0"
    "/ph/mean/60_100/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
)

polaris_url_oglala = polaris_url_format.format(
    min_lat=min_lat, min_lon=min_lon, max_lat=max_lat, max_lon=max_lon)

polaris_oglala_da = rxr.open_rasterio(polaris_url_oglala, masked=True).squeeze()

# polaris_oglala_da.hvplot(rasterize=True)*grass_gdf.hvplot()

min_lat, min_lon = 44, -100
max_lat, max_lon = min_lat+1, min_lon+1
polaris_url_format = (
    "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0"
    "/ph/mean/60_100/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
)

polaris_url_fp = polaris_url_format.format(
    min_lat=min_lat, min_lon=min_lon, max_lat=max_lat, max_lon=max_lon)

polaris_fp_da = rxr.open_rasterio(polaris_url_fp, masked=True).squeeze()
# polaris_fp_da.rio.set_spatial_dims(x_dim= 'x', y_dim= 'y', inplace=True)

# polaris_fp_da.hvplot(rasterize=True)

In [8]:
# Function for srtm download

def download_srtm_data(name, geometry):
    """
    Download SRTM raster for a given geometry, start date, and end date
    
    Downloads data from NASA Shuttle Radar Topography Mission (SRTM)
    given a spatial and temporal extent. NASA Shuttle Radar 
    Topography Mission Global 1 arc second launched 
    February 11, 2000 and ﬂew for 11 days.
    <https://lpdaac.usgs.gov/products/srtmgl1v003/>

    Parameters
    ==========
    name : str
      The name used to label the download
    grasspolygon : geopandas.GeoDataFrame
      The geometry to derive the download extent from. 
      Must have a `.envelope` attribute.

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    
    print(f'\nGRASSLANDN: {name}')
    srtm_downloader = eaapp.AppeearsDownloader(
        download_key = name.lower().replace(' ', '-'),
        product='SRTMGL1_NC.003',
        layer='SRTMGL1_DEM',
        start_date='02-11-2000',
        end_date='02-21-2000',
        polygon=geometry,
        ea_dir=os.path.join(et.io.HOME, 'earth-analytics')
    )
    if not os.path.exists(srtm_downloader.data_dir):
        srtm_downloader.submit_task_request()
        print("SRTM requested and downloading...")
        srtm_downloader.download_files()
    else:
        print("SRTM data already downloaded.")
    return srtm_downloader

srtm_paths = {}
for GRASSLANDN, grass_gs in grass_gdf.iterrows():
    print('\nGrassland name: {}'.format(GRASSLANDN))
    print(grass_gs)
    print(f"Total bounds: {grass_gs.geometry.bounds}")
    srtm_downloader = (
        download_srtm_data(GRASSLANDN, grass_gdf.loc[[GRASSLANDN]])
        )
    srtm_paths[GRASSLANDN]=(glob(
        os.path.join(
            srtm_downloader.data_dir,
            'SRTMGL1_NC.003*',
            '*.tif')
    ))


Grassland name: Oglala National Grassland
NATIONALGR                                         295521010328
GIS_ACRES                                            215804.927
SHAPE_AREA                                             0.096279
SHAPE_LEN                                              1.970612
geometry      POLYGON ((-103.72476827000003 43.0009967600000...
Name: Oglala National Grassland, dtype: object
Total bounds: (-104.05313982000001, 42.74092908, -103.36517901000002, 43.00177636000001)

GRASSLANDN: Oglala National Grassland
SRTM data already downloaded.

Grassland name: Fort Pierre National Grassland
NATIONALGR                                         281771010328
GIS_ACRES                                            209044.225
SHAPE_AREA                                             0.095149
SHAPE_LEN                                              1.455518
geometry      POLYGON ((-100.08408953000003 44.2816249699999...
Name: Fort Pierre National Grassland, dtype: object
Total bounds

In [ ]:
# Define function for opening the downloaded
# SRTM data for chosen grasslands

def load_srtm(name, srtm_paths):
    """
    Load in and merge downloaded srtm

    Parameters
    ==========
    name : str
      The name used to label the download
    srtm_paths : dict 
        path to srtm files

    Returns
    =======
    srtm_gdf: rxr.DataArray
      DataArray with the srtm data
    """
    
    print(f'\nGRASSLANDN: {name}')
    grass_srtm_da = (
        rxr.open_rasterio(srtm_paths[name][0], masked = True)
         .squeeze()
    )
    return grass_srtm_da

In [ ]:
oglala_srtm_da = load_srtm('Oglala National Grassland', srtm_paths)
fort_pierre_srtm_da = load_srtm('Fort Pierre National Grassland', srtm_paths)
oglala_srtm_da
fort_pierre_srtm_da